In [3]:
from app.core.model_builder import ModelBuilder
from sixgill.pipesim import Model
from sixgill.definitions import ModelComponents, Parameters
from app.project import get_string_values_from_class, generate_dict_from_class
from sixgill.core.model_context import ContextError
from sixgill.core.mapping import ParameterError
import pandas as pd
import logging

logger = logging.getLogger(__name__)

In [2]:
model = Model.open("trail.pips")

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 15400


INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.


In [5]:
generate_dict_from_class(Parameters).keys()

dict_keys(['About', 'AbstractTask', 'AbstractTubingSection', 'AssociatedToFluid', 'BlackOilFluid', 'BlackOilUserEmulsionTable', 'BlackOilUserViscosityTable', 'Boundary', 'CanBeTemplate', 'Casing', 'CheckValve', 'Choke', 'CoatingHeatTransfer', 'Completion', 'CompletionConingPoint', 'CompletionTestPoint', 'CompositionalFluid', 'Compressor', 'DeWaardCorrosionModel', 'DistributedCompletionModel', 'DownholeComponent', 'DownholeLocation', 'DownholeOrSurfaceComponent', 'ESP', 'ESPMotorCoefficient', 'EngineKeywords', 'Expander', 'FetkovitchEquationModel', 'FileBasedFluid', 'FlowControlValve', 'FlowCorrelation', 'Flowline', 'FlowlineGeometry', 'Fluid', 'FluidComponent', 'FluidComponentFraction', 'FluidLink', 'GLDiagnosticsSimulation', 'GasLiftInjection', 'GasliftSensitivity', 'GenericEquipment', 'GenericMultiphaseBooster', 'GeographicSurvey', 'GeothermalSurvey', 'GlobalCatalog', 'GravelPack', 'HeatExchanger', 'HeatTransferOption', 'IPRBackPressure', 'IPRDarcy', 'IPRForchheimer', 'IPRHorizontalP

In [ ]:
Parameters.